In [1]:
from random import randint
from collections import defaultdict
from uuid import uuid4
from dataclasses import dataclass
from datetime import datetime
from time import sleep
import json

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
import dma.MechaCombat as mc
QuantumSystem = mc.QuantumSystem

In [ ]:
from ipynb.fs.full.CalibrationMachine import CalibratedCaptureMachine, CalibratedCaptureConfiguration, Camera \
    hStackImages, vStackImages

In [ ]:
@dataclass
class QuantumObject(TrackedObject):
    name: str
    objectType: str

    def __eq__(self, other):
        return super().__eq__(other)

    def __repr__(self):
        changeSet = {camNum: cS for camNum, cS in self.changeSet.items() if cS is not None and cS.changeType not in [None, 'delete']}
        return f"QO - {self.objectType} - {self.name} ({changeSet})"
    
    def __post_init__(self):
        super().__post_init__()
        self.qid = str(uuid4())
        self.actionHistory = {}

    def previousVersion(self):
        return type(self)({camNum: change.lastChange if change is not None else None for camNum, change in self.changeSet.items()}, self.name, self.objectType)

In [ ]:
class HarmonyMachine(CalibratedMachine):
    states = ["idle", "unstable", "classify", "illegal"]
    modes = ["passive", "add", "move"]
            
    def __repr__(self):
        return f"Harmony -- {self.mode} - {self.state}"

    def moveMode(self):
        self.mode = "move"
    
    def addMode(self):
        self.mode = "add"

    def annotateObject(self, oid, objName, objType):
        caps = {cap.oid: cap for cap in self.newObjectBuffer + self.memory}
        cap = caps[oid]
        center = self.cc.rsc.changeSetToRealCenter(cap)
        qObj = QuantumObject(cap.changeSet, objName, objType)
        self.factories[objType](objName, "Unaligned")
        self.memory.append(qObj)
        self.newObjectBuffer.remove(cap)
        self.lastMemory = {"annotatedObject": qObj}

    def deleteObject(self, oid):
        newCaps = {cap.oid: cap for cap in self.newObjectBuffer}
        if oid in newCaps:
            cap = newCaps[oid]
            self.newObjectBuffer.remove(cap)
        memCaps = {cap.oid: cap for cap in self.memory}
        if oid in memCaps:
            cap = memCaps[oid]
            self.memory.remove(cap)
        self.lastMemory = {"deletedObject": oid}
        self.mode = "passive"

    def isLegal(self, classifiedChange):
        return True

    def cycle(self):
        cycleStart = datetime.utcnow()
        minimumClassificationTime = 1
        try:
            print(f"Starting Cycle {self.cycleCounter:5} -- {self}" + '' if self.state != "action" else f" {self.actionState}")
            self.cycleCounter += 1
            nextState = "idle"
            self.hc.capture()
            changes = self.referenceFrameDeltas()

            if self.mode == "passive" or (changes.empty and self.state == "idle"):
                self.hc.setReference()
            elif not changes.empty:
                nextState = "unstable"
                if self.state in ["unstable", "classify"] and changes == self.lastChanges:  # TODO: and cameraChangeOverlap
                    classification = self.classifyChanges(changes)
                    nextState = "classify"
                    if self.state == "classify":
                        if self.mode in ["move", "action"]:
                            assert not classification.isNewObject, f"Cannot add objects in {self.mode}"
                            if self.mode == "move":
                                assert self.isLegal(classification), f"{classificaton} is an illegal move"
                            elif self.mode == "action":
                                assert (lastDist := self.cc.rsc.trackedObjectLastDistance(classification)) < self.MM_FOR_MOVE, f"{classification} moves too far ({lastDist}) for selection"
                                assert self.actionState not in ['roll'], "Gameboard changed while monitoring for dice roll"
                            
                            nextState = "idle"
                            print("Committing classification")
                            self.commitChanges(classification)
                            if self.mode == "action":
                                if len(self.selectedObjects) == 2:
                                    self.selectedObjects = [self.lastMemory["changedObject"]]
                                    self.actionState = "sel1"
                                else:
                                    self.selectedObjects.append(self.lastMemory["changedObject"])
                                    self.actionState = "sel1" if len(self.selectedObjects) == 1 else "confirm"
                                print(f"Selected Objects: {self.selectedObjects}")
                                self.lastMemory = {"selectedObject": self.lastMemory["changedObject"]}
                            self.hc.setReference()
                        elif self.mode == "add":
                            nextState = "idle"
                            self.storeNewObject(classification)
                            self.hc.setReference()
                        elif self.mode == "calibrate":
                            try:
                                self.calibrateToObject(classification)
                            except AssertionError as ae:
                                print(f"Failed Calibration: {ae}")
                            nextState = "idle"
                            self.hc.setReference()
                        else:
                            raise Exception("Unrecognized State")
                        
                    cycleEnd = datetime.utcnow()
                    if (cycleTime := (cycleEnd - cycleStart).total_seconds()) < minimumClassificationTime:
                        sleep(minimumClassificationTime - cycleTime)
            self.state = nextState
            self.lastChanges = changes
            self.lastClassification = classification
        except Exception as e:
            from traceback import format_exc
            print("CYCLE FAILURE!!!")
            print(format_exc())
            self.state = "unstable"
            self.lastChanges = None
            self.lastClassification = None
            raise

In [ ]:
class MechaCombatMachine(HarmonyMachine):
    def __init__(self, config: CalibratedCaptureConfiguration):
        super().__init__(config)
        mc.qs.reset()
        self.factories = mc.ObjectFactories
        self.factories["Huntsman"]("Initializer", "Unaligned")
            
    def __repr__(self):
        return f"MechaCombat -- {self.mode} - {self.state}"

In [ ]:
print("Building Harmony Machine")
cc = HarmonyConfiguration()
cm = HarmonyMachine(cc)

In [ ]:
#print(f"Initialing Dice Watcher: {cm.cc.collectDiceRoll()}")

In [ ]:
if __name__ == "__main__" and False:
    cm.cycleForCalibration()
    for camName, cons in cc.rsc.converters.items():
        if len(cons) == 1:
            plt.imshow(cons[0].showUnwarpedImage())
        else:
            fig, axes = plt.subplots(1, len(cons))
            for ax, c in zip(axes, cons):
                ax.imshow(c.showUnwarpedImage())
        plt.title(f"{camName} Unwarped")
        plt.show()
    cc.saveConfiguration()

In [ ]:
if __name__ == "__main__":
    for i in range(3):
        cm.cc.capture()
    for i in range(2):
        print("Waiting for object...")
        obj = cm.cycleForAddition()
        cm.annotateObject(obj.oid, f"Foo{i}", "Huntsman")
        plt.imshow(obj.visual())
        plt.show()

In [ ]:
if __name__ == "__main__":
    plt.imshow(cm.buildMiniMap())
    plt.show()

In [ ]:
if __name__ == "__main__" and False:
    for i in range(2):
        print("Waiting for movement")
        obj = cm.cycleForMove()
        plt.imshow(obj.visual())
        plt.show()

In [ ]:
if __name__ == "__main__":
    lastMem = cm.cycleForAction()
    obj = lastMem['actedOn']
    print(obj.actionHistory)
    plt.imshow(obj.visual())
    plt.show()